In [1]:
from vector_database import *
import pandas as pd
import sqlite3
import chromadb

c:\Users\Pc\AppData\Local\pypoetry\Cache\virtualenvs\eve-project-13oD41wO-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
survey_db_path = r'C:\Users\Pc\Documents\Talk_to_Eve_mvp\documents\surveys\survey_database.db'
conn = sqlite3.connect(survey_db_path)

In [20]:
questions = pd.read_sql_query("SELECT * FROM question", conn)
answers = pd.read_sql_query("SELECT * FROM answer", conn)
conn.close()

In [21]:
answers.head(5)

answer_id_question_id  answer_id  question_id  survey_id  \
0                      1          0            0          0   
1                      2          1            0          0   
2                      3          2            0          0   
3                      4          0            1          0   
4                      5          1            1          0   

                 answer  
0  Officer / Chief / SL  
1     Operator / Rating  
2                 Other  
3              Under 24  
4               25 - 34

In [22]:
answers['answer_id'] = answers['answer_id'].astype(str)

In [23]:
answers_list = answers.groupby(['question_id', 'survey_id']).apply(lambda x: dict(zip(x['answer_id'], x['answer']))).to_list()

In [24]:
gruped_dict = answers.groupby(['question_id', 'survey_id']).apply(lambda x: dict(zip(x['answer_id'], x['answer']))).to_dict()

In [25]:
ans_tot = [{'answers': gruped_dict[(question_id, survey_id)], 'question_type': question_type} for (question_id, survey_id), question_type in zip(questions[['question_id', 'survey_id']].to_records(index=False), questions['question_type'])]

In [26]:
ans_st = [ans_tot[i]['answers']|{'question_type':ans_tot[i]['question_type']} for i in range(len(ans_tot))]

In [10]:
ans_st

[{'0': 'Officer / Chief / SL',
  '1': 'Operator / Rating',
  '2': 'Other',
  'question_type': 'radio'},
 {'0': 'Under 24',
  '1': '25 - 34',
  '2': '35 - 44',
  '3': '45 - 54',
  '4': 'Over 55',
  'question_type': 'radio'},
 {'0': '00 - 12',
  '1': '12 - 24',
  '2': 'Dayshift',
  '3': 'Other',
  'question_type': 'radio'},
 {'0': '1 year or less',
  '1': '2 - 4 years',
  '2': '5 - 9 years',
  '3': '10 - 15 years',
  '4': 'Over 15 years',
  'question_type': 'radio'},
 {'0': 'Tethys',
  '1': 'Sovereign',
  '2': 'Vanguard',
  '3': 'Atlas',
  '4': 'Hyperion',
  '5': 'Tansa',
  '6': 'Apollo',
  '7': 'Swift',
  '8': 'Victory',
  'question_type': 'radio'},
 {'0': 'Less than 2 hrs',
  '1': '3 - 4 hrs',
  '2': '5 - 7 hrs',
  '3': '8 - 11 hrs',
  '4': 'Over 11 hrs',
  'question_type': 'radio'},
 {'0': 'Never',
  '1': 'Almost never',
  '2': 'Sometimes',
  '3': 'Fairly often',
  '4': 'Very often',
  'question_type': 'radio'},
 {'0': 'Never',
  '1': '0 - 60 minutes',
  '2': '61 - 100 minutes',
  '3'

In [31]:
# new column with a super id (survey_id_question_id)
questions['super_id'] = questions['survey_id'].astype(str) + '_' + questions['question_id'].astype(str)
questions.head(19)

question_id  survey_id                                           question  \
0             0          0                             What is your position?   
1             1          0                                  What is your age?   
2             2          0                             What shift are you on?   
3             3          0  How many years of seismic offshore experience ...   
4             4          0                       Which vessel do you work on?   
5             5          0  How many hours per day (on - and off shift) do...   
6             6          0  How often do you go outside to get some fresh ...   
7             7          0  How often do you do exercise in your time off ...   
8             8          0  How many servings of fruit and vegetables do y...   
9             9          0  How many days a week do you usually eat fast f...   
10           10          0  Healthy options that I want to eat are served ...   
11           11          0  When was the last time you had your cholestero...   
12           12          0  When was the last time you had your blood pres...   
13           13          0  I’m comfortable seeing the Medic if I feel unw...   
14           14          0  In the last month, how often have you felt tha...   
15           15          0  In the last month, how often have you felt con...   
16           16          0  In the last month, how often have you felt tha...   
17           17          0  In the last month, how often have you felt dif...   
18           18          0  How often are you able to stay in touch with y...   

   question_type super_id  
0          radio      0_0  
1          radio      0_1  
2          radio      0_2  
3          radio      0_3  
4          radio      0_4  
5          radio      0_5  
6          radio      0_6  
7          radio      0_7  
8          radio      0_8  
9          radio      0_9  
10         radio     0_10  
11         radio     0_11  
12         radio     0_12  
13         radio     0_13  
14         radio     0_14  
15         radio     0_15  
16         radio     0_16  
17         radio     0_17  
18         radio     0_18

In [28]:
chroma_client = chromadb.PersistentClient(path='../../documents/surveys')

In [95]:
#chroma_client.delete_collection(name="survey_questions")

In [13]:
collection = chroma_client.create_collection(name="survey_questions")

In [14]:
collection.add(documents=questions['question'].values.tolist(),
               metadatas=ans_st,
                ids=questions['super_id'].values.tolist())

In [15]:
results = collection.query(
    query_texts=["experience"],
    n_results=2
)

In [134]:
results

{'ids': [['0_14', '0_16']],
 'distances': [[1.2705490560220394, 1.3907022887846388]],
 'metadatas': [[{'0': 'Never',
    '1': 'Almost never',
    '2': 'Sometimes',
    '3': 'Fairly often',
    '4': 'Very often',
    'question_type': 'radio'},
   {'0': 'Never',
    '1': 'Almost never',
    '2': 'Sometimes',
    '3': 'Fairly often',
    '4': 'Very often',
    'question_type': 'radio'}]],
 'embeddings': None,
 'documents': [['In the last month, how often have you felt that you were unable to control the important things in your life?',
   'In the last month, how often have you felt that things were going yourway?']],
 'uris': None,
 'data': None}

In [143]:
a = [{int(k) if k.isdigit() else k: v for k, v in d.items()} for d in results['metadatas'][0]]


In [147]:
a = [{k: v for k, v in d.items() if not k.isdigit()} for d in results['metadatas'][0]]


In [149]:
a[0]['question_type']

'radio'

## conv flow

In [35]:
db_tst_path = r'C:\Users\Pc\Documents\Talk_to_Eve_mvp\users\botChuck\botChuck.db'
conn = sqlite3.connect(db_tst_path)
session_history = pd.read_sql_query('SELECT * from sessions_history', con=conn)
conn.close()

In [25]:
print(session_history.iloc[0]['message'])

Ahoy there, Eve. I reckon I'm feeling a bit worn down today, to be honest. It's just been one of those rough patches at sea, you know? The long hours and isolation are really starting to take a toll on me.


In [26]:
results = collection.query(
    query_texts=[session_history.iloc[0]['message']],
    n_results=2
)

In [75]:
import pandas as pd

In [112]:
counts = pd.DataFrame(columns=['survey_id', 'question_id', 'count', 'dist', 'min_dist'])
# make id the index
counts.set_index(['survey_id', 'question_id'], inplace=True)
for i in range(len(session_history)):
    results = collection.query(
    query_texts=[session_history.iloc[i]['message']],
    n_results=2
)
    for j in range(2):
        current_id = tuple(map(int, results['ids'][0][j].split('_')))
        if current_id not in counts.index:
            counts.loc[current_id, :] = {'count': 1, 'dist': results['distances'][0][j], 'min_dist': results['distances'][0][j]}
        else:
            counts.at[current_id, 'count'] += 1
            counts.at[current_id, 'dist'] = (counts.at[current_id, 'dist'] + results['distances'][0][j]) / 2
            counts.at[current_id, 'min_dist'] = min(counts.at[current_id, 'min_dist'], results['distances'][0][j])


In [113]:
counts

count      dist  min_dist
survey_id question_id                          
0         16            8.0  1.408668  0.980793
          17            6.0  1.193282  0.979257
          14           15.0  1.278577    0.9534
          19            9.0   1.26237  1.024239
          5             3.0  1.186154  1.103214
          18           10.0   1.39297   1.08929
          26            3.0  1.382178  1.265623
          15            1.0   1.43317   1.43317
          30            1.0  1.396893  1.396893
          27            5.0  1.285143  1.089573
          23            3.0  1.375913  1.204266
          22            2.0  1.482097  1.353381
          24            1.0  1.257089  1.257089
          21            1.0  1.303814  1.303814
          4             1.0  1.440331  1.440331
          3             3.0  1.374677  1.254379
          0             4.0  1.546052  1.451355

In [96]:
# df query count>10 and min_dist<0.9
len(counts.query('count>10 & min_dist<0.9'))

0

In [114]:
# Ejemplo de dos DataFrames
df1 = pd.DataFrame({
    'survey_id': [1, 1, 2, 2],
    'question_id': [1, 2, 1, 2],
    'count': [10, 5, 8, 3],
    'dist': [0.5, 0.8, 0.3, 0.6],
    'min_dist': [0.2, 0.4, 0.1, 0.5]
})

df2 = pd.DataFrame({
    'survey_id': [1, 1, 2],
    'question_id': [1, 2, 1],
    'answer_id': ['A', 'B', 'C']
})

# Combinar los DataFrames usando merge
merged_df = pd.merge(df1, df2, on=['survey_id', 'question_id'], how='left')

# Filtrar las filas donde answer_id no es nulo
result_df = merged_df[merged_df['answer_id'].isnull()]

# Mostrar el resultado
print(result_df)

   survey_id  question_id  count  dist  min_dist answer_id
3          2            2      3   0.6       0.5       NaN


In [100]:
tuple('0_1'.split('_'))

('0', '1')

In [31]:
counts = {}

In [40]:
counts[results['ids'][0][0]] = {'count':1, 'dist': results['distances'][0][0]}

In [44]:
counts['0_16']['count'] += 1

In [45]:
counts['0_16']['count']

2

In [59]:
import sys
import os
from IPython.display import display, Markdown

sys.path.append(r'C:\Users\Pc\Documents\Talk_to_Eve_mvp')
from configurations.config_llm import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [60]:
# Importa la biblioteca de OpenAI
from openai import OpenAI

def obtener_respuesta_gpt4(prompt, historial=[]):
    """
    Realiza una solicitud a la API de OpenAI utilizando el modelo GPT-4.
    
    Args:
        prompt (str): El texto de entrada o pregunta para el modelo.
    
    Returns:
        str: La respuesta generada por el modelo.
    """
    try:
        # Crea una instancia del cliente OpenAI
        cliente = OpenAI()

        messages=[{"role": "user", "content": prompt}]
        if len(historial) > 0:
            # xtend messages list role: changing roles user, agent. the none indexes are agent the pairs are user
            messages.extend([{"role": "user", "content": m} if i % 2 != 0 else {"role": "assistant", "content": m} for i, m in enumerate(historial)])

        # Realiza una solicitud de chat completions con el modelo GPT-4
        respuesta = cliente.chat.completions.create(
            model="gpt-4",  # Utiliza el modelo GPT-4
            messages=messages
        )

        # Devuelve la respuesta generada por el modelo
        return respuesta.choices[0].message
    except Exception as e:
        return f"Error al obtener respuesta: {str(e)}"

In [72]:
q = questions.iloc[6]['question']
prompt = f"""I'm developing a chatbot that incorporates a function to bring up relevant survey questions during a conversation based on a vectorized database. The goal is to enhance the user experience by seamlessly integrating survey questions when relevant. The chatbot utilizes a Language Model (LLM) for natural language understanding.

I have a database containing survey questions. I want to create embeddings for use to compare the ongoing conversation with the stored questions. If a similarity threshold is met, the chatbot introduces one of the relevant survey questions into the conversation.

To facilitate this process, you want the LLM to convert the survey questions into conversational phrases. This transformation is crucial for effective comparison with user input. Below is an example prompt:

Could you please help me convert the following survey question into a conversational phrase, not a question. To embed this phrase and then compare with user input to determine if the bot can ask a question related to the survey question.
---

{q}
"""
response = obtener_respuesta_gpt4(prompt)
display(Markdown(response.content))

I usually step out to enjoy some fresh air several times a day.

In [73]:
q

'How often do you go outside to get some fresh air during a day?'

In [29]:
chroma_client = chromadb.PersistentClient('../../documents/surveys')
collection = chroma_client.get_collection("survey_questions")

In [4]:
results = collection.query(
    query_texts=["experience"],
    n_results=2
)

In [16]:
def tst(a):
    if a:
        return None
    else:
        return {'a': 'pene'}

In [18]:
if tst(True):
    print('a')

In [33]:
questions.query('question_id==14')['question'].values[0]

'In the last month, how often have you felt that you were unable to control the important things in your life?'

In [34]:
questions.query('question_id==18')['question'].values[0]

'How often are you able to stay in touch with your family and friends while offshore?'

In [37]:
session_history['message'].values[0]

"Ahoy there, Eve. I reckon I'm feeling a bit worn down today, to be honest. It's just been one of those rough patches at sea, you know? The long hours and isolation are really starting to take a toll on me."